In [3]:
from dataClass import DataClass
import pandas as pd

directory = "stock_market_data/sp500/csv"
tickers_csv = "tickers_list.csv"
tickers_df = pd.read_csv(tickers_csv)
# Initialize with your directory and tickers list
data_loader = DataClass(
    directory=directory,
    tickers_csv=tickers_csv
)

# Load the Adjusted Close dataframe
adj_close_df = data_loader.load_adjusted_close()

In [4]:
print(adj_close_df.shape[1])

171


In [5]:
missing_tickers = set(tickers_df.iloc[:, 0].astype(str).str.upper()) - set(adj_close_df.columns[1:])
print(missing_tickers)
print(len(missing_tickers))

{'WFC', 'CMA', 'NUE', 'PPL', 'KLAC', 'NKE', 'MAS', 'GLW', 'WMT', 'PFG', 'PGR', 'INTC', 'HIG', 'AES', 'GS', 'ETN', 'L', 'BF-B', 'DHR', 'CMCSA', 'UNH', 'CI', 'CMS', 'CCL', 'LLY', 'SPGI', 'ADBE', 'DUK', 'AFL', 'ETR', 'ORCL', 'COF', 'NVDA', 'EXC', 'MS', 'CSX', 'PNC', 'CVS', 'IPG', 'SYY', 'SNA', 'TGT', 'NEM', 'QCOM', 'AIG', 'CVX', 'ALL', 'DVN'}
48


In [6]:
import yfinance as yf

# List of missing tickers
missing_tickers = set(tickers_df.iloc[:, 0].astype(str).str.upper()) - set(adj_close_df.columns[1:])

# Download Adjusted Close for all missing tickers at once (vectorized)
if missing_tickers:
    data = yf.download(
        list(missing_tickers),
        start=adj_close_df['Date'].min(),
        end=adj_close_df['Date'].max(),
        progress=True,
        auto_adjust=False,
    )['Adj Close']
    if isinstance(data, pd.Series):
        data = data.to_frame()
    data = data.reset_index()
    # Merge with existing adj_close_df on 'Date'
    adj_close_df = pd.merge(adj_close_df, data, on='Date', how='outer')
    adj_close_df = adj_close_df.sort_values('Date').reset_index(drop=False)
else:
    print('No missing tickers to download.')

[*********************100%***********************]  48 of 48 completed


In [7]:
print(data.shape)
print(adj_close_df.shape)

print(data)
print(adj_close_df)

(13355, 49)
(13356, 220)
Ticker       Date        ADBE        AES        AFL        AIG         ALL  \
0      1970-01-02         NaN        NaN        NaN        NaN         NaN   
1      1970-01-05         NaN        NaN        NaN        NaN         NaN   
2      1970-01-06         NaN        NaN        NaN        NaN         NaN   
3      1970-01-07         NaN        NaN        NaN        NaN         NaN   
4      1970-01-08         NaN        NaN        NaN        NaN         NaN   
...           ...         ...        ...        ...        ...         ...   
13350  2022-12-05  334.089996  25.516567  67.572411  58.206039  121.487038   
13351  2022-12-06  331.149994  25.300701  67.619972  58.357388  123.255791   
13352  2022-12-07  326.679993  25.165791  67.724586  58.679020  123.066628   
13353  2022-12-08  332.579987  25.489584  66.887543  58.896603  122.990959   
13354  2022-12-09  330.640015  25.651482  66.545128  59.133095  121.591087   

Ticker       BF-B   CCL          CI   